In [1]:
# %%

# import math # roundは組み込み関数のため不要
# jax関連のライブラリを全てインポート
import jax.numpy as jnp
import matplotlib.pyplot as plt
from jax import vmap

# from LikelihoodEvaluator import LikelihoodEvaluator
# from ParameterEstimator import ParameterEstimator
from sympy import Array, symbols

from degenerate_sim import (
    DegenerateDiffusionProcess,
    LikelihoodEvaluator,
    m_estimate,
    newton_solve,
    m_estimate_jax,
    bayes_estimate
)
from degenerate_sim.evaluation.likelihood_evaluator_jax import SymbolicPreparationError

x_sym, y_sym = symbols("x, y")
theta_10, theta_20, theta_21 = symbols("theta_10 theta_20 theta_21")
theta_30, theta_31 = symbols("theta_30 theta_31")

# ベクトル化（1次元テンソルとして）
x = Array([x_sym])
y = Array([y_sym])
theta_1 = Array([theta_10])
theta_2 = Array([theta_20, theta_21])
theta_3 = Array([theta_30])

# Step 2: A, B, H の式定義
A = Array([-theta_20*x_sym  - theta_21*y_sym])
B = Array([[theta_10]])
H = Array([(theta_30 * x_sym)])

# インスタンス
FNmodel = DegenerateDiffusionProcess(
    x=x,
    y=y,
    theta_1=theta_1,
    theta_2=theta_2,
    theta_3=theta_3,
    A=A,
    B=B,
    H=H,
)
FN_likelihood = LikelihoodEvaluator(FNmodel)

true_theta1 = jnp.array([1.0])
true_theta2 = jnp.array([1.0, 1.0])
true_theta3 = jnp.array([1.0])
true_theta = (true_theta1, true_theta2, true_theta3)
t_max = 10
burn_out = 50
h = 0.2
bounds_theta1 = [(0.1, 2.0)]
bounds_theta2 = [(0.1, 2.0), (0.1, 2.0)]
bounds_theta3 = [(0.1, 2.0)]
n = t_max//h
x_series, y_series = FNmodel.simulate(true_theta=true_theta, t_max=t_max, burn_out=burn_out, h=h)


In [ ]:
import importlib
from pathlib import Path
import numpy as np
import degenerate_sim.estimation.loop_estimation_algorithm as loop_algo_mod
loop_algo_mod = importlib.reload(loop_algo_mod)
from degenerate_sim.estimation.loop_estimation_algorithm import (
    LoopEstimationAlgorithm,
    SimulationConfig,
    format_stage_comparison_latex,
    format_summary_as_latex,
    save_loop_results_to_csv,
    save_summary_to_csv,
    summarize_loop_results,
 )

loop_plan = {
    1: ("B", "B", "B"),
    2: ("M", "M", "M"),
    3: ("M", "M", "M"),
    4: ("M", "M", "M"),
    5: ("M", "M", "M"),
    6: ("M", "M", "M"),
    7: ("M", "M", "M"),
    
}

loop_sim_cfg = SimulationConfig(
    true_theta=true_theta,
    t_max=100.0,
    h=0.05,
    burn_out=50.0,
    dt=0.01,
 )

loop_estimator = LoopEstimationAlgorithm(
    evaluator=FN_likelihood,
    simulation_config=loop_sim_cfg,
    bounds_theta1=bounds_theta1,
    bounds_theta2=bounds_theta2,
    bounds_theta3=bounds_theta3,
 )

loop_results = loop_estimator.run(
    seeds=range(500),
    plan=loop_plan,
    k_0=5,
    initial_theta_stage0=(
        jnp.array([0.2]),
        jnp.array([0.5, 0.5]),
        jnp.array([0.2, 0.1]),
    ),
    my_setting=True,
    show_progress=True,
    progress_desc="Seeds",
    num_workers=8,

 )




Seeds:   0%|          | 0/10 [00:00<?, ?seed/s]

KeyboardInterrupt: 

In [ ]:
import importlib
from pathlib import Path
import degenerate_sim.estimation.parameter_estimator as parameter_estimator_mod
import degenerate_sim.estimation.loop_estimation_algorithm as loop_algo_mod

parameter_estimator_mod = importlib.reload(parameter_estimator_mod)
loop_algo_mod = importlib.reload(loop_algo_mod)
from degenerate_sim.estimation.loop_estimation_algorithm import (
    build_output_directory,
    format_stage_comparison_latex,
    format_summary_as_latex,
    save_loop_results_to_csv,
    save_model_structure,
    save_summary_to_csv,
    summarize_loop_results,
 )

summary_stats = summarize_loop_results(loop_results, ddof=1)

num_steps = int(round(loop_sim_cfg.t_max / loop_sim_cfg.h))
nh_value = float(loop_sim_cfg.h * num_steps)
metadata = {
    "h": float(loop_sim_cfg.h),
    "nh": nh_value,
}

model_label = "linear_model"

output_dir = build_output_directory(
    Path("experiments"),
    model_label,
    h=metadata["h"],
    nh=metadata["nh"],
)

summary_csv_path = save_summary_to_csv(
    summary_stats,
    output_dir / "summary.csv",
    true_theta=true_theta,
    metadata=metadata,
)
raw_csv_path = save_loop_results_to_csv(
    loop_results,
    output_dir / "detail.csv",
)
model_structure_path = save_model_structure(
    FNmodel,
    output_dir,
    filename="model_structure.md",
)

caption_summary = "FN model final-phase summary"
latex_table_summary = format_summary_as_latex(
    summary_stats,
    phase="final",
    true_theta=true_theta,
    caption=caption_summary,
)

caption_comparison = "Stage-0 vs Final comparison for FN model"
latex_table_comparison = format_stage_comparison_latex(
    summary_csv_path,
    phases=("stage0", "final"),
    caption=caption_comparison,
)

print(f"Summary CSV saved to: {summary_csv_path}")
print(f"Detail CSV saved to: {raw_csv_path}")
print(f"Model structure saved to: {model_structure_path}")
print()
print(latex_table_summary)
print()
print(latex_table_comparison)